# Programación declarativa @ URJC
# Programación funcional
## Curso 21-22, convocatoria ordinaria (27/29 de octubre de 2021)
## Campus de Vicálvaro/Móstoles


# Definiciones auxiliares

In [1]:
import $ivy.`org.scalatest::scalatest:3.0.8`
import org.scalatest._

import $ivy.$                               

import org.scalatest._

### Algunas funciones sobre tipos estándar de la librería de Scala

In [2]:
// type Tree[A] = 1 + Tree[A] * A * Tree[A]

sealed abstract class Tree[A]
case class Empty[A]() extends Tree[A]
case class Node[A](left: Tree[A], root: A, right: Tree[A]) extends Tree[A]

def foldTree[A, B](tree: Tree[A])(empty: B)(node: (B, A, B) => B): B = 
    tree match {
        case Empty() => empty
        case Node(left, root, right) => node(foldTree(left)(empty)(node), root, foldTree(right)(empty)(node))
    }

defined class Tree
defined class Empty
defined class Node
defined function foldTree

The companion object defines some smart constructors that will allow us to write test cases more easily.

In [3]:
object Tree{
    
    def void[A]: Tree[A] = 
        Empty()
    
    def leaf[A](a: A): Node[A] = 
        Node(Empty(), a, Empty())
    
    def right[A](a: A, tree: Tree[A]): Node[A] = 
        Node(Empty(), a, tree)
    
    def left[A](tree: Tree[A], a: A): Node[A] = 
        Node(tree, a, Empty())
    
    def node[A](left: Tree[A], a: A, right: Tree[A]): Node[A] = 
        Node(left, a, right)
}

import Tree._

defined object Tree
import Tree._

In [4]:
object Signatures{
    abstract class List[A]{
        
        // Common HOFs
        def foldRight[B](directSol: B)(composeSol: (A, B) => B): B
        def foldLeft[B](initial: B)(update: (B, A) => B): B
        def map[B](f: A => B): List[B]
        def flatMap[B](f: A => List[B]): List[B]
        def filter(f: A => Boolean): List[A]
        def forall(pred: A => Boolean): Boolean
        def exists(pred: A => Boolean): Boolean
        
        // Reverse a list
        // e.g. List(1,2,3).reverse==List(3,2,1)
        def reverse: List[A]
        
        // Take the first `n` elements of the list
        // e.g. List(1,2,3).take(2) == List(1,2)
        //      List(1,2,3).take(0) == List()
        //      List(1,2,3).take(5) == List(1,2,3)
        def take(n: Int): List[A]
        
        // Drop the first `n` elements of the list 
        // e.g. List(1,2,3).drop(2) == List(3)
        //      List(1,2,3).drop(0) == List(1,2,3)
        //      List(1,2,3).drop(4) == List()
        def drop(n: Int): List[A]

        // List concatenation
        // e.g. List(1,2,3).concat(List(4,5)) == List(1,2,3,4,5)
        def concat(l: List[A]): List[A]
    }
    
    abstract class Option[A]{
        // Test whether the value is defined (i.e. `Some`) or not (i.e. `None`)
        def isDefined: Boolean 
        def map[B](f: A => B): Option[B]
    }
    
    abstract class Either[A, B]{
        // Test whether the value is left or right
        def isLeft: Boolean 
        def isRight: Boolean 
        def map[C](f: B => C): Either[A, C]
    }
}

defined object Signatures

### Definiciones auxiliares sobre la correspondencia Curry-Howard

In [5]:
type Not[P] = P => Nothing
type <=>[P, Q] = (P => Q, Q => P)
type Or[P, Q] = Either[P, Q]
type And[P, Q] = (P, Q)

defined type Not
defined type <=>
defined type Or
defined type And

# Ejercicio 1 (variante 1)


__a) (2 puntos)__ Utiliza la correspondencia de Curry-Howard para demostrar que los siguientes razonamientos de la lógica proposicional representan deducciones válidas de la lógica intuicionista: 

* $\neg p \rightarrow \neg\neg\neg p$

In [65]:
def proof[P]: Not[P] => Not[Not[Not[P]]] = 
    (notP: P => Nothing) => 
        (f: (P => Nothing) => Nothing) => 
            f(notP) : Nothing

defined function proof

__b) (1 punto)__ Utiliza la correspondencia de Curry-Howard para demostrar el siguiente teorema de la lógica clásica: 

$(\neg q \rightarrow \neg p) \rightarrow (p \rightarrow q)$

Supóngase para ello que la ley del tercio excluso se cumple para la variable proposicional $q$, es decir, que la fórmula $q \vee \neg q$  puede utilizarse como premisa.


In [62]:
def proof[P, Q](middle: Either[Q, Not[Q]]): (Not[Q] => Not[P]) => (P => Q) = 
    (f: Not[Q] => Not[P]) => (p: P) => 
        middle match {
            case Left(q: Q) => 
                q : Q
            case Right(notQ: Not[Q]) => 
                f(notQ)(p) : Q
        }

defined function proof

# Ejercicio 1 (variante 2)


__a) (2 puntos)__ Utiliza la correspondencia de Curry-Howard para demostrar que los siguientes teoremas de la lógica intuicionista: 

* Dilema constructivo complejo: $⊢ (p ∨ q) → (p→r) → (q→s) → (r ∨ s)$

In [70]:
def proof[P, Q]: (P, Q) => Not[Either[Not[P], Not[Q]]] = 
    { case (p: P, q: Q) => {
        case Left(notP: (P => Nothing)) => notP(p)
        case Right(notQ: (Q => Nothing)) => notQ(q)
    }}

defined function proof

In [111]:
def proof[P, Q]: Either[P, Q] => Not[(Not[P], Not[Q])] = 
    { 
        case Left(p: P) => 
            { case (notP, notQ) => notP(p) }
        case Right(q: Q) => 
            { case (notP, notQ) => notQ(q) }
    }

defined function proof

In [83]:
def proof[P]: Not[Not[Either[P, Not[P]]]] = 
    (notPorNotP: Either[P, Not[P]] => Nothing) => 
        notPorNotP(Right((p: P) => 
                         notPorNotP(Left(p)) : Nothing): Either[P, Not[P]]): Nothing

defined function proof

In [84]:
def proof[P]: Not[Not[Either[P, Not[P]]]] = 
    e => e(Right(p => e(Left(p))))

defined function proof

__b) (1 punto)__ Utiliza la correspondencia de Curry-Howard para demostrar el siguiente teorema de la lógica clásica: 

$⊢ ((p →q) →p)→p$

Supóngase para ello que la ley del tercio excluso se cumple para la variable proposicional $p$, es decir, que la fórmula  $p \vee \neg p$  puede utilizarse como premisa.

In [69]:
def proof[P, Q](middle: Either[P, Not[P]]): ((P => Q) => P) => P = 
    (f: (P => Q) => P) => 
        middle match {
            case Left(p: P) => 
                p
            case Right(notP: Not[P]) => 
                f((p: P) => notP(p) : Q) : P 
        }

defined function proof

In [115]:
def proof[P, Q](middleP: Either[P, Not[P]], middleQ: Either[Q, Not[Q]]): Not[(Not[P], Not[Q])] => Either[P, Q] =  
    (middleP, middleQ) match {
        case (Left(p), _) => 
            _ => Left(p)
        case (_, Left(q)) => 
            _ => Right(q)
        case (Right(notP), Right(notQ)) => 
            _((notP, notQ))
    }
        

defined function proof

# Ejercicio 2 (v)
__(1 punto)__

Demuestra el siguiente isomorfismo entre tipos algebraicos de datos para todo tipo $X$: 

$(1+1)^X \cong Boolean^X$

de tal forma que se verifique el siguiente test unitario para $X=Int$:

_(variante 1)_

In [148]:
class IsoTest(
    from: Either[Int, Either[Unit, Unit] => Int] => (Int, Option[Int]), 
    to: ((Int, Option[Int])) => Either[Int, Either[Unit, Unit] => Int]
) extends FlatSpec with Matchers{
    
    "from-to" should "work" in {
        from(to((0, None))) shouldBe (0, None)
        from(to((0, Some(1)))) shouldBe (0, Some(1))
    }
    
    val f: Either[Unit, Unit] => Int = {
        case Left(()) => 0
        case Right(()) => 1
    }
    
    "to-from" should "work" in {
        to(from(Left(0))) shouldBe Left(0)
        to(from(Right(f))) shouldBe Right(f)
    }
}

defined class IsoTest

In [152]:
def from[X](l: Either[X, Either[Unit, Unit] => X]): (X, Option[X]) =
    l match {
        case Left(x) => (x, None)
        case Right(f) => (f(Left(())), Some(f(Right(()))))
    }

defined function from

In [153]:
def to[X](l: (X, Option[X])): Either[X, Either[Unit, Unit] => X] =
    l match {
        case (x, None) => Left(x)
        case (x1, Some(x2)) => Right{
            case Left(()) => x1
            case Right(()) => x2
        }
    }

defined function to

In [154]:
run(new IsoTest(from[Int], to[Int]))

cmd147$Helper$IsoTest:
from-to
- should work
to-from
- should work *** FAILED ***
  Right(ammonite.$sess.cmd152$Helper$$Lambda$3396/0x00000008018ab040@771d26d2) was not equal to Right(ammonite.$sess.cmd147$Helper$IsoTest$$Lambda$3394/0x00000008018aa040@4ffebd00) (cmd147.sc:18)



_(variante 2)_

# Ejercicio 2 
__(1 punto)__

Demuestra el siguiente isomorfismo entre tipos algebraicos de datos para todo tipo $X$: 

$(1+1)^X \cong Boolean^X$

de tal forma que se verifique el siguiente test unitario para $X=Int$:

_(variante 1)_

In [107]:
class IsoTest(
    from: (Int => Either[Unit, Unit]) => Int => Boolean, 
    to: (Int => Boolean) => Int => Either[Unit, Unit]
) extends FlatSpec with Matchers{
    
    val f: Int => Either[Unit, Unit] = 
        i => if (i % 2 == 0) Left(()) else Right(())
    
    val g: Int => Boolean = 
        _ % 2 == 0
    
    "from-to" should "work" in {
        from(to(g))(0) shouldBe g(0)
        from(to(g))(1) shouldBe g(1)
        from(to(g))(2) shouldBe g(2)
        from(to(g))(3) shouldBe g(3)
    }
    
    "to-from" should "work" in {
        to(from(f))(0) shouldBe f(0)
        to(from(f))(1) shouldBe f(1)
        to(from(f))(2) shouldBe f(2)
        to(from(f))(3) shouldBe f(3)
    }
}

defined class IsoTest

In [104]:
def from[X](l: X => Either[Unit, Unit]): X => Boolean = 
    x => l(x) match {
        case Left(()) => true
        case Right(()) => false
    }

defined function from

In [105]:
def to[X](l: X => Boolean): X => Either[Unit, Unit] = 
    x => if (l(x)) Left(()) else Right(())

defined function to

In [106]:
run(new IsoTest(from[Int], to[Int]))

cmd102$Helper$IsoTest:
from-to
- should work
to-from
- should work


_(variante 2)_

# Ejercicio 2 
__(1 punto)__

Demuestra el siguiente isomorfismo entre tipos algebraicos de datos para todo tipo $X$: 

$(1+1)^X \cong Boolean^X$

de tal forma que se verifique el siguiente test unitario para $X=Int$:

_(variante 1)_

In [107]:
class IsoTest(
    from: (Int => Either[Unit, Unit]) => Int => Boolean, 
    to: (Int => Boolean) => Int => Either[Unit, Unit]
) extends FlatSpec with Matchers{
    
    val f: Int => Either[Unit, Unit] = 
        i => if (i % 2 == 0) Left(()) else Right(())
    
    val g: Int => Boolean = 
        _ % 2 == 0
    
    "from-to" should "work" in {
        from(to(g))(0) shouldBe g(0)
        from(to(g))(1) shouldBe g(1)
        from(to(g))(2) shouldBe g(2)
        from(to(g))(3) shouldBe g(3)
    }
    
    "to-from" should "work" in {
        to(from(f))(0) shouldBe f(0)
        to(from(f))(1) shouldBe f(1)
        to(from(f))(2) shouldBe f(2)
        to(from(f))(3) shouldBe f(3)
    }
}

defined class IsoTest

In [104]:
def from[X](l: X => Either[Unit, Unit]): X => Boolean = 
    x => l(x) match {
        case Left(()) => true
        case Right(()) => false
    }

defined function from

In [105]:
def to[X](l: X => Boolean): X => Either[Unit, Unit] = 
    x => if (l(x)) Left(()) else Right(())

defined function to

In [106]:
run(new IsoTest(from[Int], to[Int]))

cmd102$Helper$IsoTest:
from-to
- should work
to-from
- should work


_(variante 2)_

# Ejercicio 3 (variante 1)
__(3 puntos)__

La función de orden superior `sequence` recibe una lista de valores opcionales y devuelve una lista con todos los valores pertenecientes a la lista de entrada (en el mismo orden), _en caso de que todos los valores de la lista de entrada estén definidos_; si alguno de los valores opcionales de la lista de entrada es `None`, entonces la función sequence devuelve `None` también. El comportamiento de la función se ilustra en el siguiente test unitario:


In [128]:
class TestIndex(
    index: List[Char] => Int => Option[Char]
) extends FlatSpec with Matchers{
    "index" should "work" in {
        index(List())(0) shouldBe None
        index(List())(1) shouldBe None
        index(List())(2) shouldBe None
        
        index(List('a'))(0) shouldBe Some('a')
        index(List('a'))(1) shouldBe None
        index(List('a'))(2) shouldBe None
        
        index(List('a','b'))(0) shouldBe Some('a')
        index(List('a','b'))(1) shouldBe Some('b')
        index(List('a','b'))(2) shouldBe None
    }
}

defined class TestIndex

__a) (1 punto)__ Implementa la función `lift` de manera recursiva.

In [129]:
def index[A](l: List[A]): Int => Option[A] = 
    l match {
        case Nil => 
            _ => None
        case head :: tail => 
            idx => if (idx == 0) Some(head)
                    else index(tail)(idx-1)
    }

defined function index

In [130]:
run(new TestIndex(index))

cmd127$Helper$TestIndex:
index
- should work


__b) (1 punto)__ Implementa la función `lift` utilizando `foldRight`.

In [127]:
def index[A](l: List[A]): Int => Option[A] = 
    l.foldRight((_: Int) => None: Option[A]){
        (head, tailIndex) => 
            idx => if (idx == 0) Some(head)
                   else tailIndex(idx-1)
    }

defined function index

In [131]:
run(new TestIndex(index))

cmd127$Helper$TestIndex:
index
- should work


__c) (1 punto)__ con foldLeft

In [135]:
def index[A](l: List[A]): Int => Option[A] = 
    l.foldLeft((0, (_: Int) => None: Option[A])){
        case ((next, out), e) => 
            (next + 1, idx => 
                 if (idx < next) out(idx) 
                 else if (idx == next) Some(e) 
                 else None)
    }._2

defined function index

In [133]:
run(new TestIndex(index))

cmd127$Helper$TestIndex:
index
- should work


# Ejercicio 3 (variante 2)
__(3 puntos)__

La función de orden superior `sequence` recibe una lista de valores de tipo `X` o `Y` y devuelve una lista con los valores de tipo `Y` pertenecientes a la lista de entrada (en el mismo orden), _en caso de que dicha lista no contenga ningún valor de tipo `X`_; en caso de que sí lo contenga, la función `sequence` devolverá el primer valor de tipo `X` encontrado. El comportamiento de la función se ilustra en el siguiente test unitario, donde la función `sequence` se encuentra particularizada para los tipos `X=String` e `Y=Int`:


In [137]:
class TestSlice(
    slice: List[Int] => (Int, Int) => List[Int]
) extends FlatSpec with Matchers{
    "slice" should "work" in {
        slice(List())(0,3) shouldBe List()
        slice(List(1,2,3,4))(5,6) shouldBe List()
        slice(List(1,2,3,4))(0,2) shouldBe List(1,2,3)
        slice(List(1,2,3,4))(0,6) shouldBe List(1,2,3,4)
        slice(List(1,2,3,4))(1,3) shouldBe List(2,3,4)
        slice(List(1,2,3,4))(1,2) shouldBe List(2,3)
    }
}

defined class TestSlice

__a) (1 punto)__ Implementa la función `slice` con recursión final.

In [138]:
def slice[A](list: List[A])(from: Int, to: Int): List[A] = {
    def auxSlice(aux: List[A])(out: List[A], idx: Int): List[A] = 
        aux match {
            case head :: tail if from <= idx && idx <= to => 
                auxSlice(tail)(head :: out, idx + 1)
            case _ :: tail if idx < from => 
                auxSlice(tail)(out, idx + 1)
            case _ => 
                out.reverse
        }
    
    auxSlice(list)(Nil, 0)
}

defined function slice

In [140]:
run(new TestSlice(slice))

cmd136$Helper$TestSlice:
slice
- should work


__b) (1 punto)__ Implementa la función `slice` con `foldLeft`.

In [141]:
def slice[A](list: List[A])(from: Int, to: Int): List[A] = 
    list.foldLeft((Nil: List[A], 0)){
        case ((out: List[A], idx: Int), e) if from <= idx && idx <= to => 
            (e :: out, idx + 1)
        case ((out, idx), _) => 
            (out, idx + 1)
    }._1.reverse

defined function slice

In [142]:
run(new TestSlice(slice))

cmd136$Helper$TestSlice:
slice
- should work


# Ejercicio 4a
__(3 puntos)__

Hallar el camino más largo entre la raíz y las hojas de un árbol binario. 

In [41]:
class TestLongestPath(longest: Tree[Int] => List[Int]) extends FlatSpec with Matchers{
    "longest path" should "work" in {
        longest(void) shouldBe List()
        longest(left(left(right(3,right(2,leaf(1))), 4), 5)) shouldBe List(5,4,3,2,1)
        longest(node(left(leaf(4), 1), 0, node(leaf(3), 2, right(2, right(4, leaf(5)))))) shouldBe List(0, 2, 2, 4, 5)
    }
}

defined class TestLongestPath

__a) (1,5 puntos)__ Recursivamente.

In [42]:
def longestPath[A, B](tree: Tree[A]): List[A] = 
    tree match {
        case Empty() => List()
        case Node(left, root, right) => 
            val longestLeft: List[A] = longestPath(left)
            val longestRight: List[A] = longestPath(right)
            root :: (if (longestLeft.length > longestRight.length) longestLeft else longestRight)
    }

defined function longestPath

In [43]:
run(new TestLongestPath(longestPath))

cmd40$Helper$TestLongestPath:
longest path
- should work


__b) (1,5 puntos)__ Con foldTree.

In [44]:
def longestPath[A, B](tree: Tree[A]): List[A] = 
    foldTree(tree)(List[A]())(
        (longestLeft, root, longestRight) => 
            root :: (if (longestLeft.length > longestRight.length) longestLeft else longestRight)
    )

defined function longestPath

Sin utilizar la función `length`

In [51]:
def longestPath[A, B](tree: Tree[A]): List[A] = 
    foldTree(tree)((0, List[A]())){
        case ((l, longestLeft), root, (r, longestRight)) => 
            if (l > r) (l+1, root :: longestLeft) 
            else (r+1, root :: longestRight)
    }._2

defined function longestPath

In [50]:
run(new TestLongestPath(longestPath))

cmd40$Helper$TestLongestPath:
longest path
- should work


_(variante 1)_

# Ejercicio 4b
__(3 puntos)__

Hallar el camino más largo entre la raíz y las hojas de un árbol binario. 

In [52]:
class TestHighestCostPath(highest: Tree[Int] => List[Int]) extends FlatSpec with Matchers{
    "highest path" should "work" in {
        highest(void) shouldBe List()
        highest(left(left(right(3,right(2,leaf(1))), 4), 5)) shouldBe List(5,4,3,2,1)
        highest(node(left(leaf(8), 7), 0, node(leaf(3), 2, right(2, right(4, leaf(5)))))) shouldBe List(0, 7, 8)
    }
}

defined class TestHighestCostPath

In [53]:
def sum(list: List[Int]): Int = 
    list.foldRight(0)(_ + _)

defined function sum

__a) (1,5 puntos)__ Recursivamente.

In [54]:
def highestPath(tree: Tree[Int]): List[Int] = 
    tree match {
        case Empty() => List()
        case Node(left, root, right) => 
            val highestLeft: List[Int] = highestPath(left)
            val highestRight: List[Int] = highestPath(right)
            root :: (if (sum(highestLeft) > sum(highestRight)) highestLeft else highestRight)
    }

defined function highestPath

In [55]:
run(new TestHighestCostPath(highestPath))

cmd51$Helper$TestHighestCostPath:
highest path
- should work


__b) (1,5 puntos)__ Con foldTree.

In [56]:
def highestPath(tree: Tree[Int]): List[Int] = 
    foldTree(tree)(List[Int]())(
        (highestLeft, root, highestRight) => 
            root :: (if (sum(highestLeft) > sum(highestRight)) highestLeft else highestRight)
    )

defined function highestPath

Sin utilizar la función `sum`

In [59]:
def highestPath(tree: Tree[Int]): List[Int] = 
    foldTree(tree)((0, List[Int]())){
        case ((l, highestLeft), root, (r, highestRight)) => 
            if (l > r) (l+root, root :: highestLeft) 
            else (r+root, root :: highestRight)
    }._2

defined function highestPath

In [60]:
run(new TestHighestCostPath(highestPath))

cmd51$Helper$TestHighestCostPath:
highest path
- should work


_(variante 1)_